# Continuous Integration using Jenkins

### Building a local Jenkins environment

Using the installation instructions for Docker at https://jenkins.io/doc/book/installing/, create a local Docker Compose environment:

Create a Dockerfile called `Dockerfile.dind` for Docker-in-Docker that allows Jenkins to run jobs as the Jenkins user (uid 1000).  The DIND entrypoint script will detect a group called docker on startup, and grant group ownership rights to `/var/run/docker.sock`.

In [ ]:
FROM docker:dind
LABEL application=jenkins

# Create docker group
RUN addgroup -g 1000 docker

Create a Dockerfile called `Dockerfile.agent` that will provide a Docker environment to run Jenkins builds.

The agent only requires make, docker client and docker compose.

In [ ]:
FROM alpine
LABEL application=jenkins

RUN apk add --no-cache python3-dev musl-dev libffi-dev openssl-dev gcc bash git make coreutils curl
RUN curl -fs -L https://download.docker.com/linux/static/stable/x86_64/docker-19.03.8.tgz | tar xvfz - --strip-components 1 -C /usr/local/bin docker/docker
RUN pip3 install docker-compose

Create a simple Makefile to automate various tasks for building and starting the Jenkins environment:

Build and publish the agent image by running `make agent`:

In [ ]:
$ make agent
...
...

Start up environment by running `make jenkins`.  The output should show initial admin password:

Browse to http://localhost:8080 and enter the initial admin password.

When asked to install plugins, choose customise option and ensure the GitHub plugin is selected.

After installation is completed, downgrade the GitHub API plugin as per https://issues.jenkins-ci.org/browse/JENKINS-61822?focusedCommentId=388713&page=com.atlassian.jira.plugin.system.issuetabpanels%3Acomment-tabpanel#comment-388713:

- Go to http://updates.jenkins-ci.org/download/plugins/github-api/
- Download 1.106 version plugin
- Install via Manage Jenkins > Manage Plugins > Advanced > Upload Plugin

### Create a new pipeline

Use BlueOcean wizard to create an initial pipeline that runs:

- make test
- make release
- make clean